In [166]:
from sqlalchemy import create_engine
from rent_scrapper import Arkadia_scrapper,push_newest_data
import pandas as pd
import matplotlib.pyplot as plt

In [167]:
#push_newest_data()

In [168]:
config = { 'user': "root",
            'password': "rootroot",
            'host': "localhost",
            'database': "apartments",
            'port': '3306'}

In [176]:
def create_url(user, password, host, database, port):
    return f"mysql://{user}:{password}@{host}:{port}/{database}"

pool = create_engine(url=create_url(**config),
                     pool_size=20, max_overflow=0
                     )
with pool.connect() as conn:
    sql = """SELECT m.room_number ,
m.apartment ,
date_update ,
price_floor ,
price_ceil ,
Avaliable_date ,
`Sq.Ft` ,
Floor_Plan ,
num_bedroom ,
num_bathroom
FROM apartments.prices p
	LEFT JOIN apartments.room_meta m
		ON p.apartment = m.apartment AND p.room_number = m.room_number;"""
    df = pd.read_sql(sql, conn)

In [177]:
df = df.merge(df.groupby(['Sq.Ft','date_update']).price_floor.mean().reset_index().rename({'price_floor':'price_mean_horizontal'},axis=1),
         on = ['Sq.Ft','date_update'])
df = df.merge( df.groupby('room_number').price_floor.mean().reset_index().rename({'price_floor':'price_mean_vertical'},axis=1),
               on = 'room_number')

In [178]:
df.dtypes

apartment                        object
date_update              datetime64[ns]
price_floor                     float64
price_ceil                      float64
Avaliable_date                   object
Sq.Ft                            object
Floor_Plan                       object
num_bedroom                      object
num_bathroom                     object
price_mean_horizontal           float64
room_number                      object
price_mean_vertical             float64
dtype: object

In [179]:
df['deviation_from_cohort'] = df.eval('price_floor - price_mean_horizontal')
df['deviation_from_history'] = df.eval('price_floor - price_mean_vertical')
df['price_sqft'] = df.eval('price_floor / `Sq.Ft`')

TypeError: unsupported operand type(s) for /: 'float64' and 'object'

In [ ]:
# Analyze best deals
df.dropna(subset='price_floor').query('price_floor < price_mean_horizontal').sort_values(['room_number','date_update','price_mean_horizontal'])

In [ ]:
# Analyze best deals
df.dropna(subset='price_floor').query('price_floor < price_mean_vertical').sort_values(['room_number','date_update','deviation_from_history'])

In [ ]:
# Understand trend
df.query('date_update == "{}"'.format(df.date_update.max())).deviation_from_history.hist(figsize=(16,9))

In [ ]:
def show_history(df_floor_group,title,by='price_floor'):
    legends = []
    plt.figure(figsize=(20,10))
    for group,df_group in df_floor_group.groupby('room_number'):
        legends.append(group+'|'+str(df_group['Sq.Ft'].to_list()[-1]))
        df_group = df_group.set_index('date_update')
        plt.plot(df_group.index, df_group[by])
    for group,df_group in df_floor_group.groupby('room_number'):
        #legends.append(group+'|'+str(df_group['Sq.Ft'].to_list()[-1]))
        df_group = df_group.set_index('date_update')
        plt.scatter(df_group.index, df_group[by])
    plt.legend(legends,loc='lower left', bbox_to_anchor=(1.01, 0., 0.5, 0.5))
    plt.title(title)
    plt.show()

In [ ]:
df.query('date_update == "{}"'.format(df.date_update.max())).sort_values('price_sqft')

In [ ]:
df_plot = df.query('num_bedroom == "1" and num_bathroom == "1" and price_floor < 2600 and `Sq.Ft` <'
                   '= 799')#.sort_values('price_floor',ascending=False)
###
df_plot.date_update = pd.DatetimeIndex(df_plot.date_update)
show_history(df_plot,'ALL','price_sqft')
show_history(df_plot,'ALL')
for Floor_Plan,df_floor_group in df_plot.groupby('Floor_Plan'):
    show_history(df_floor_group,Floor_Plan)